## Libraries

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from llama_index.core import SimpleDirectoryReader
from langchain_community.document_loaders import DirectoryLoader

## Creating chunks

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

dynamic_path = f'./data/'
loader = DirectoryLoader(dynamic_path, glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
splits = text_splitter.split_documents(documents=documents)


Created a chunk of size 1325, which is longer than the specified 1000
Created a chunk of size 1860, which is longer than the specified 1000
Created a chunk of size 2502, which is longer than the specified 1000
Created a chunk of size 2590, which is longer than the specified 1000
Created a chunk of size 1111, which is longer than the specified 1000
Created a chunk of size 1245, which is longer than the specified 1000
Created a chunk of size 1107, which is longer than the specified 1000


## Creating Embeddings if not created

In [7]:
embedding = OpenAIEmbeddings()

persist_directory = 'docs/chroma/'


vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
) 

# save the database so we can use it later
vectordb.persist()

# check that the database have been created and get the number of documents
print(vectordb._collection.count())

/Users/swapnildhamu/Developer/vin-ai/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


312


## Similarity search with score 

In [8]:
# similarity search
question = "how can i setup auto label printing"


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)



0.27672719955444336
5. **Install QZ Tray**:
   - Run the installation file downloaded in the previous step.
   - Ensure that Java SE Development Kit is installed on your machine.

6. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

7. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that "Automatically Start" option is enabled.

### Step 4: Label Printing

8. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.

---

These detailed instructions cover the setup process for auto label printing using a Zebra printer. Following these steps ensures smooth and efficient label printing operations.

For further assistance, refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


0.2812996506690979
Title: Zebra Printer Settings for Auto Printing

Modified 

## Loading existing vectorDB (If embeddings are already created) and querying

In [9]:
persist_directory = 'docs/chroma/'
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

result = qa_chain({"query": question})
print(result["result"])

/Users/swapnildhamu/Developer/vin-ai/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/swapnildhamu/Developer/vin-ai/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install the Zebra Printer Driver or a ZPL supported printer driver on your machine.
   - Rename the Zebra printer as "VinLBL" in the Devices and Printers section.

3. **QZ Tray Installation**:
   - Download the QZ Tray software from [QZ Tray GitHub](https://github.com/qzind/tray/tags) or [QZ Tray Official Website](https://qz.io/download/).
   - Install QZ Tray by running the downloaded installation file and ensure that Java SE Development Kit is installed on your machine.

4. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

5. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

6. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.


## Using templates to form a chain and get specific answer

In [11]:
template = """Address the user as vinner and Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

print(result["result"])

To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install the Zebra Printer Driver or a ZPL supported printer driver on your machine.
   - Rename the Zebra printer as "VinLBL" in the Devices and Printers section.

3. **QZ Tray Installation**:
   - Download the QZ Tray software from [QZ Tray GitHub](https://github.com/qzind/tray/tags) or [QZ Tray Official Website](https://qz.io/download/).
   - Install QZ Tray by running the downloaded installation file and ensure that Java SE Development Kit is installed on your machine.

4. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

5. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

6. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.


## Using memory to store the conversation and get the answer

In [23]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb.as_retriever(),
    memory=memory
)
question = "how can i setup auto label printing"
result = qa({"question": question})

print(result['answer'])

To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install the Zebra Printer Driver or a ZPL supported printer driver on your machine.
   - Rename the Zebra printer as "VinLBL" in the Devices and Printers section.

3. **QZ Tray Installation**:
   - Download the QZ Tray software from [QZ Tray GitHub](https://github.com/qzind/tray/tags) or [QZ Tray Official Website](https://qz.io/download/).
   - Install QZ Tray by running the downloaded installation file and ensure that Java SE Development Kit is installed on your machine.

4. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

5. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

6. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.


## ChatHistory

In [24]:
question = "Can you provide me the link for the same which i can refer to"
result = qa({"question": question})
print(result['answer'])

Sure, you can find detailed instructions on setting up auto label printing with a Zebra printer by following this link: [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).
